In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# Set random seed for reproducibility
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
def generate_matrices(n_samples, n, condition_number=10):
    A_matrices = []
    A_inv_matrices = []
    
    for _ in range(n_samples):
        # Generate a random matrix
        U, _, Vt = np.linalg.svd(np.random.randn(n, n))
        s = np.linspace(1, condition_number, n)
        S = np.diag(s)
        A = U @ S @ Vt
        
        # Compute the inverse
        A_inv = np.linalg.inv(A)
        
        A_matrices.append(A)
        A_inv_matrices.append(A_inv)
    
    return np.array(A_matrices), np.array(A_inv_matrices)

n_samples = 1000
matrix_size = 5

A_matrices, A_inv_matrices = generate_matrices(n_samples, matrix_size)

In [3]:
def build_model(matrix_size):
    # Define inputs
    A_input = Input(shape=(matrix_size, matrix_size), name='A_input')
    X_input = Input(shape=(matrix_size, matrix_size), name='X_input')
    
    # Flatten the inputs
    A_flat = tf.keras.layers.Flatten()(A_input)
    X_flat = tf.keras.layers.Flatten()(X_input)
    
    # Concatenate the inputs
    concatenated = Concatenate()([A_flat, X_flat])
    
    # Dense layers for refinement
    hidden = Dense(128, activation='relu')(concatenated)
    hidden = Dense(128, activation='relu')(hidden)
    
    # Output layer
    delta_X_flat = Dense(matrix_size * matrix_size)(hidden)
    delta_X = tf.keras.layers.Reshape((matrix_size, matrix_size))(delta_X_flat)
    
    # Define the model
    model = Model(inputs=[A_input, X_input], outputs=delta_X)
    
    return model

model = build_model(matrix_size)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 A_input (InputLayer)           [(None, 5, 5)]       0           []                               
                                                                                                  
 X_input (InputLayer)           [(None, 5, 5)]       0           []                               
                                                                                                  
 flatten (Flatten)              (None, 25)           0           ['A_input[0][0]']                
                                                                                                  
 flatten_1 (Flatten)            (None, 25)           0           ['X_input[0][0]']                
                                                                                              

In [ ]:
def train_model(model, A_matrices, A_inv_matrices, epochs=100, batch_size=32):
    optimizer = Adam(learning_rate=0.001)
    mse_loss = MeanSquaredError()

    for epoch in range(epochs):
        # Shuffle the data
        indices = np.arange(len(A_matrices))
        np.random.shuffle(indices)
        A_matrices = A_matrices[indices]
        A_inv_matrices = A_inv_matrices[indices]
        
        for i in range(0, len(A_matrices), batch_size):
            A_batch = A_matrices[i:i + batch_size]
            A_inv_batch = A_inv_matrices[i:i + batch_size]
            
            # Initialize the approximation with the identity matrix
            X_approx = np.tile(np.eye(matrix_size), (len(A_batch), 1, 1))
            
            # Perform iterative refinement
            for _ in range(10):  # Number of refinement steps
                with tf.GradientTape() as tape:
                    delta_X = model([A_batch, X_approx])
                    X_approx += delta_X
                    loss = mse_loss(A_batch @ X_approx, np.tile(np.eye(matrix_size), (len(A_batch), 1, 1)))
                
                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            
        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.numpy()}')

train_model(model, A_matrices, A_inv_matrices)

In [5]:
# Generate test data
n_test_samples = 100
A_test, A_inv_test = generate_matrices(n_test_samples, matrix_size)

def evaluate_model(model, A_test, A_inv_test):
    X_approx = np.tile(np.eye(matrix_size), (len(A_test), 1, 1))
    
    for _ in range(10):
        delta_X = model.predict([A_test, X_approx])
        X_approx += delta_X
    
    mse_loss = MeanSquaredError()
    test_loss = mse_loss(A_test @ X_approx, np.tile(np.eye(matrix_size), (len(A_test), 1, 1)))
    print(f'Test Loss: {test_loss.numpy()}')

evaluate_model(model, A_test, A_inv_test)

4/4 [==============================] - 0s 1ms/step
Test Loss: 1.05301112189169
